In [1]:
## Librerías Básicas
import pandas as pd
import numpy as np
from datetime import datetime

## Librerías para Warnings
import warnings
warnings.filterwarnings('ignore')

## Librerías para Scrapping
from selenium import webdriver
import time
from time import sleep

In [2]:
## Declaramos path y leemos archivo de urls
path = "C:/Users/mibra/Desktop/NBA/"

## Loading urls data
urls = pd.read_csv(path+"url_nba_teams.csv", sep=";")

## Fixing the date
date = time.strftime("%Y-%m-%d")

## Obtain the predicted games for today
games = pd.read_csv(path+"Black_Mamba_Classifier_Pred_Future_games.csv", sep=";")
games = games.loc[games.date==date]

## Path para acceder a chromedriver
path_to_chromedriver = path+'chromedriver.exe' 

## Chrome automatizado
browser = webdriver.Chrome(executable_path=path_to_chromedriver)

FileNotFoundError: [Errno 2] File C:/Users/Miguel/Desktop/NBA/url_nba_teams.csv does not exist: 'C:/Users/Miguel/Desktop/NBA/url_nba_teams.csv'

In [3]:
## Url of the betting web
url = "https://www.888sport.es/baloncesto/#/filter/basketball/nba"

## Get daily nba games
try:
    browser.get(url)
    sleep(10)
except:
    sleep(5)
    browser.get(url)
    sleep(10)

## Get betting odds
all_games = pd.DataFrame()
for game in range(1, len(games)+1):          
    try:                           
        table = browser.find_element_by_xpath('//*[@id="KambiBC-content"]/div/div/div/div[4]/div[1]/div/div[3]/div[2]/div/div/div[2]/div/ul/li[{}]/a/div/div[2]/div/div/div/div'.format(game))
        sleep(2)
    except:
        sleep(2)
        table = browser.find_element_by_xpath('//*[@id="KambiBC-content"]/div/div/div/div[4]/div[1]/div/div[3]/div[2]/div/div/div[2]/div/ul/li[{}]/a/div/div[2]/div/div/div/div'.format(game))
        sleep(2)
    table = table.text.replace("Los Angeles Clippers", "LA Clippers").split("\n")
    table = pd.DataFrame({"team":table[::2], "odds":table[1::2]})
    all_games = pd.concat([all_games, table], axis=0)

## Merge with urls data
all_games = all_games.merge(urls.drop(["url", "section"], axis=1), left_on="team", right_on="team")
games = games.merge(all_games.drop(["team"], axis=1), left_on="team1", right_on="codigo", how="left")
games = games.merge(all_games.drop(["team"], axis=1), left_on="team2", right_on="codigo", how="left")
games = games.drop(["date", "codigo_x", "codigo_y"], axis=1)

## Rename and order the columns
games.columns = ["match_up", "pred_team1", "team1", "team2", "pred_team2", "diff", "odds_team1", "odds_team2"]
games = games[["match_up", "team1", "team2", "pred_team1", "pred_team2", "diff", "odds_team1", "odds_team2"]]

## Obtaining the expected probabilities
games["prob_1"] = [round(100/float(i), 2) for i in games.odds_team1]
games["prob_2"] = [round(100/float(i), 2) for i in games.odds_team2]

## Obtaining the Expected Value
games["EV%_1"] = (games.odds_team1.astype("float64")*games.pred_team1*100-100).round(2)
games["EV%_2"] = (games.odds_team2.astype("float64")*games.pred_team2*100-100).round(2)

## Applying rules to bet
games["betting"] = "not bet"
games.loc[(games["EV%_1"] > 10) & (games["pred_team1"] > games["pred_team2"]), "betting"] = "team1_l2"
games.loc[(games["EV%_1"] > 10) & (games["pred_team1"] < games["pred_team2"]), "betting"] = "team1_l1"        
games.loc[(games["EV%_2"] > 10) & (games["pred_team2"] > games["pred_team1"]), "betting"] = "team2_l2"
games.loc[(games["EV%_2"] > 10) & (games["pred_team2"] < games["pred_team1"]), "betting"] = "team2_l1"

## Obtaining stake
games["stake"] = 0
games.loc[(games["betting"] == "team1_l2") | (games["betting"] == "team2_l2") , "stake"] = (2/(games.loc[(games["betting"] == "team1_l2") | (games["betting"] == "team2_l2")].odds_team1.astype("float64") - 1)).round(2)
games.loc[(games["betting"] == "team1_l1") | (games["betting"] == "team2_l1") , "stake"] = (1/(games.loc[(games["betting"] == "team1_l1") | (games["betting"] == "team2_l1")].odds_team2.astype("float64") - 1)).round(2)
           
## Saving historic results
historic = pd.read_csv(path+"Black_Mamba_Classifier_Bettting_Future_games.csv", sep=";", header=0)
historic = pd.concat([historic, games], axis=0)
historic.to_csv(path+"Black_Mamba_Classifier_Historic_Bettting.csv", sep=";", , header=True, index=False)

## Saving results
games.to_csv(path+"Black_Mamba_Classifier_Bettting_Future_games.csv", sep=";", header=True, index=False)

In [4]:
games

,match_up,team1,team2,pred_team1,pred_team2,diff,odds_team1,odds_team2,prob_1,prob_2,EV%_1,EV%_2,betting,stake
0,"Mar 11, 2020 - NYK@ATL",NYK,ATL,0.40,0.51,-0.11,2.70,1.48,37.04,67.57,8.00,-24.52,not bet,0.00
1,"Mar 11, 2020 - MIA@CHA",MIA,CHA,0.73,0.25,0.48,1.14,6.10,87.72,16.39,-16.78,52.50,team2_l1,0.20
2,"Mar 11, 2020 - DEN@DAL",DEN,DAL,0.44,0.56,-0.12,1.79,2.08,55.87,48.08,-21.24,16.48,team2_l2,2.53
3,"Mar 11, 2020 - PHI@DET",PHI,DET,0.52,0.14,0.38,1.13,6.25,88.50,16.00,-41.24,-12.50,not bet,0.00
4,"Mar 11, 2020 - SAC@NOP",SAC,NOP,0.37,0.32,0.05,2.02,1.84,49.50,54.35,-25.26,-41.12,not bet,0.00
5,"Mar 11, 2020 - UTA@OKC",UTA,OKC,0.33,0.40,-0.07,2.25,1.68,44.44,59.52,-25.75,-32.80,not bet,0.00
